### Importing Libraries and Data Cleaning

In [49]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.preprocessing import LabelEncoder


# importing the data
derby_data = pd.read_csv('nyra_2019_complete.csv')
derby_data.head()

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,track_id,race_date,race_number,program_number,trakus_index,latitude,longitude,distance_id,course_type,track_condition,run_up_distance,race_type,purse,post_time,weight_carried,jockey,odds
0,AQU,2019-01-01,9,6,72,40.672902,-73.827607,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090
1,AQU,2019-01-01,9,6,73,40.672946,-73.827587,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090
2,AQU,2019-01-01,9,6,74,40.672990,-73.827568,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090
3,AQU,2019-01-01,9,6,63,40.672510,-73.827781,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090
4,AQU,2019-01-01,9,6,64,40.672553,-73.827762,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090


In [54]:
# Our race_type column has a few types of races that need to be removed from our dataset.
## SOURCE : https://www.equibase.com/newfan/codes.cfm

# These races are Waiver Maiden Claiming, Waiver Claiming, and Starter Handicap.
# Waiver Maiden Claiming = Horses that have never won a race that can be claimed after the race
# Waiver Claiming = Horses can race without the possibility of being claimed after coming off an extended layoff
# Starter Handicap = Horses running in an inferior claiming company, restricted to horses that have run in a claiming race

# dropping rows from dataset where race_type = WMC, WCL, or SHP
derby_data = derby_data.drop(derby_data[derby_data['race_type'] == 'WMC'].index)
derby_data = derby_data.drop(derby_data[derby_data['race_type'] == 'WCL'].index)
derby_data = derby_data.drop(derby_data[derby_data['race_type'] == 'SHP'].index)



### Feature Engineering

In [27]:
# converting date columns into datetime format
derby_data['race_date'] = pd.to_datetime(derby_data['race_date'], format = '%Y-%m-%d')

# extracting year, month, and day for analysis later
derby_data['year'] = derby_data['race_date'].dt.year
derby_data['month'] = derby_data['race_date'].dt.month
derby_data['day'] = derby_data['race_date'].dt.day

# verifying results
derby_data.head()

,track_id,race_date,race_number,program_number,trakus_index,latitude,longitude,distance_id,course_type,track_condition,run_up_distance,race_type,purse,post_time,weight_carried,jockey,odds,year,month,day
0,AQU,2019-01-01,9,6,72,40.672902,-73.827607,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090,2019,1,1
1,AQU,2019-01-01,9,6,73,40.672946,-73.827587,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090,2019,1,1
2,AQU,2019-01-01,9,6,74,40.672990,-73.827568,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090,2019,1,1
3,AQU,2019-01-01,9,6,63,40.672510,-73.827781,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090,2019,1,1
4,AQU,2019-01-01,9,6,64,40.672553,-73.827762,600,D,GD,48,CLM,25000,420,120,Andre Shivnarine Worrie,2090,2019,1,1


In [28]:
# converting track_id to numerical
# SOURCES: https://stackoverflow.com/questions/38088652/pandas-convert-categories-to-numbers
#          https://stackoverflow.com/questions/39475187/how-to-speed-labelencoder-up-recoding-a-categorical-variable-into-integers/39503973#39503973
# Chose to use this method because it is more efficient than using LabelEncoder

# encoding track_id to numerical values
derby_data['track_id_number'] = derby_data['track_id'].astype('category').cat.codes 

# verifying results
track_id_encoding = derby_data.loc[:, ['track_id', 'track_id_number']] 
# print track_id_encoding table with only one of each track_id
track_id_encoding.drop_duplicates(subset = 'track_id', keep = 'first', inplace = True) 
track_id_encoding

,track_id,track_id_number
0,AQU,0
572,BEL,1
1125,SAR,2


In [29]:
# encoding jockey to numerical values
derby_data['jockey_number'] = derby_data['jockey'].astype('category').cat.codes 

# verifying results
jockey_encoding = derby_data.loc[:, ['jockey', 'jockey_number']] 
# print jockey_encoding table with only one of each jockey name
jockey_encoding.drop_duplicates(subset = 'jockey', keep = 'first', inplace = True) 
jockey_encoding

,jockey,jockey_number
0,Andre Shivnarine Worrie,5
297,Manuel Franco,119
572,Dylan Davis,42
1125,Irad Ortiz Jr.,66
1378,Samuel Jimenez,149
...,...,...
5214475,Jozef Bojko,99
5215687,Andrea Atzeni,6
5216762,Stephane Pasquier,156
5219542,Sam Twiston-Davies,147


In [56]:
# encoding race_type to numerical values
derby_data['race_type_number'] = derby_data['race_type'].astype('category').cat.codes 

# verifying results
race_type_encoding = derby_data.loc[:, ['race_type', 'race_type_number']] 
# print race_type_encoding table with only one of each race_type 
race_type_encoding.drop_duplicates(subset = 'race_type', keep = 'first', inplace = True) 
race_type_encoding


,race_type,race_type_number
0,CLM,2
572,AOC,1
1378,ALW,0
9410,SST,6
11808,STK,7
55634,STR,8
87034,SOC,5
416637,MCL,3
478967,MSW,4


### Data Visualization and Exploratory Analysis

In [57]:
# The first thing I want to do is recheck the correlation of each variable with odds

# sorted correlation of odds with the rest of our variables
derby_data.corr()['odds'].sort_values(ascending = True)

# The strongest correlation is weight_carried. This makes sense because if the horse is carrying more weight, it will be harder for it to run quickly or for long periods of time.

weight_carried     -0.113439
longitude          -0.065728
latitude           -0.035904
distance_id        -0.023112
trakus_index       -0.002601
purse               0.005484
race_type_number    0.006017
run_up_distance     0.017226
post_time           0.021743
race_number         0.078675
odds                1.000000
Name: odds, dtype: float64